In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

In [2]:
from bs4 import BeautifulSoup
import requests
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_content = requests.get(url).text
soup = BeautifulSoup(html_content,'lxml')
#print(soup.prettify()) # print the parsed data of html

In [3]:
target_table = soup.find("table", attrs={"class": "wikitable sortable"})
target_table_data = target_table.tbody.find_all("tr")  
# Get all the headings of Lists
headings = []
for td in target_table_data[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.text.replace('\n', ' ').strip())

print(headings)

['Postcode', 'Borough', 'Neighbourhood']


In [9]:
target_table_data = target_table.tbody.find_all("tr")  
table_data=[]
for i in range(1,len(target_table_data)):
    data=[]
    for td in target_table_data[i].find_all("td"):
    # remove any newlines and extra spaces from left and right
        data.append(td.text.replace('\n', ' ').strip())
    table_data.append(data)

In [10]:
import pandas as pd
data = pd.DataFrame(table_data, columns = headings)
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
# Get names of indexes for which borough  not assigned
indexNames = data[ data['Borough'] == 'Not assigned' ].index
# Delete these row indexes from dataFrame
data.drop(indexNames , inplace=True)
data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [12]:
data.set_index('Postcode', inplace=True)

In [13]:
#data.groupby('Postcode')['Neighbourhood'].apply(' ,'.join).reset_index()
new_data= pd.DataFrame(data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(' ,'.join).reset_index())


In [114]:
new_data

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park"
7,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge"
8,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff ,Cliffside West"


In [14]:
print(new_data.shape)

(103, 3)


In [15]:
!wget -O geo_data.csv http://cocl.us/Geospatial_data

--2020-03-15 15:02:57--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.86, 158.85.108.83, 169.48.113.194
Connecting to cocl.us (cocl.us)|158.85.108.86|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-03-15 15:03:00--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-03-15 15:03:00--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-03-15 15:03:01--  https://ibm.box.com/public

In [16]:
df = pd.read_csv('geo_data.csv')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df = new_data.join(df.set_index('Postal Code'), on='Postal Code')
print(df)

KeyError: 'Postal Code'